# FINAL

In [130]:
#!g1.1

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.ops as ops
import torchvision.models as models

import os
from PIL import Image
from IPython.display import clear_output
from tqdm.notebook import tqdm

import pandas as pd
import numpy as np

import csv


In [131]:
#!g1.1
import wandb
wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: tutugarin. Use `wandb login --relogin` to force relogin


True

In [132]:
#!g1.1

DEVICE = torch.device('cuda')
DEVICE

device(type='cuda')

## Напишем класс для работы с фотографиями:

In [133]:
#!g1.1

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, root: str, transform: transforms.Compose = None,
            image_dirname: str = 'trainval/trainval', labels=None) -> None:
        super().__init__()
        self._root: str = root
        self._image_dirname: str = image_dirname

        self._transform: transforms.Compose = transform
        if self._transform is None:
            self._transform = transforms.Compose([
                transforms.Resize(224),
                transforms.ToTensor(),
            ])

        self._labels = labels
        
    def __len__(self) -> int:
        return len(self._labels)

    def __getitem__(self, item):
        label, filename = self._labels.iloc[item]['Label'], self._labels.iloc[item]['Id']
        image = Image.open(os.path.join(self._root, self._image_dirname, filename)).convert('RGB')
        image = self._transform(image)
        return image, label

## Посчитаем статистики на фотографиях:


In [134]:
#!g1.1

N_CHANNELS = 3

root = '/home/jupyter/mnt/datasets/bhw1'
train_image_dirname = 'trainval/trainval'
test_image_dirname = 'test/test'
labels_filename = 'labels.csv'

In [ ]:
#!g1.1
labels = pd.read_csv(f'{root}/{labels_filename}')
full_dataset = ImageDataset(root, image_dirname=train_image_dirname, labels=labels)
full_dataloader = torch.utils.data.DataLoader(full_dataset, batch_size=32, shuffle=False)

mean = torch.zeros(N_CHANNELS)
std = torch.zeros(N_CHANNELS)
for images, _ in tqdm(full_dataloader):
    for i in range(N_CHANNELS):
        mean[i] += images[:,i,:,:].mean()
        std[i] += images[:,i,:,:].std()

mean.div_(len(full_dataloader))
std.div_(len(full_dataloader))
print(f'{mean=}, {std=}')

## Напишем функцию, которая будет делить наш датасет и возвращать два даталоадера: для трейна и для вала

In [135]:
#!g1.1

def get_loaders(root: str, train_transform: transforms.Compose = None,
        test_size: float = 0.3, batch_size: int = 64,
        image_dirname: str = 'trainval/trainval', labels_filename: str = 'labels.csv'):
    
    test_transformer = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5696, 0.5450, 0.4936], [0.2430, 0.2375, 0.2555]),
    ])
    
    labels = pd.read_csv(f'{root}/{labels_filename}')
    train_idx, valid_idx = torch.utils.data.random_split(
        np.arange(len(labels)), 
        [1 - test_size, test_size], 
        generator=torch.Generator().manual_seed(42)
    )
    
    trainset = ImageDataset(root, train_transform, image_dirname, labels.iloc[train_idx.indices])
    valset = ImageDataset(root, test_transformer, image_dirname, labels.iloc[valid_idx.indices])

    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=8)
    val_loader = torch.utils.data.DataLoader(valset, batch_size=batch_size, shuffle=False, num_workers=8)
    
    return train_loader, val_loader

## Объявим наши аугментации:

In [136]:
#!g1.1

transform = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomPerspective(distortion_scale=0.1, p=0.5),
    transforms.RandomCrop((224, 224)),
    
    transforms.RandomHorizontalFlip(p=0.5),
    
    transforms.RandomApply([
        transforms.GaussianBlur(kernel_size=5),
    ], p=0.3),
    
    transforms.ToTensor(),
    transforms.Normalize([0.5696, 0.5450, 0.4936], [0.2430, 0.2375, 0.2555]),
    
    transforms.RandomErasing(),
])

train_loader, val_loader = get_loaders(root='/home/jupyter/mnt/datasets/bhw1',  batch_size=32, train_transform=transform, test_size=0.2)

In [137]:
#!g1.1

len(train_loader) / len(val_loader)

4.0

## Напишем функции для обучения:

In [143]:
#!g1.1

def test(model, loader, loss_func):
    loss_log = []
    acc_log = []
    model.eval()
    
    for data, target in loader:
        data = data.to(DEVICE)
        target = target.to(DEVICE)

        with torch.no_grad():
            logits = model(data)
            loss = loss_func(logits, target)
        loss_log.append(loss.item())
        
        acc = torch.sum((logits.argmax(dim=1) == target).to(torch.float32))
        acc_log.append(acc.item()) 
        
    return np.mean(loss_log), np.sum(acc_log) / len(loader.dataset)


def train_epoch(model, optimizer, train_loader, loss_func, scheduler):
    loss_log = []
    acc_log = []
    model.train()
    
    pbar = tqdm(train_loader)
    for data, target in pbar:
        data = data.to(DEVICE)
        target = target.to(DEVICE)
        
        optimizer.zero_grad()
        logits = model(data)
        loss = loss_func(logits, target)
        loss.backward()
        optimizer.step()
        
#         if scheduler is not None:
#             scheduler.step()
        
        loss_log.append(loss.item())
        
        acc = torch.sum((logits.argmax(dim=1) == target).to(torch.float32))
        acc_log.append(acc.item())

    return np.mean(loss_log), np.sum(acc_log) / len(train_loader.dataset)


def train(model, optimizer, n_epochs, train_loader, val_loader, scheduler, loss_func):
    train_loss_log, train_acc_log, val_loss_log, val_acc_log = [], [], [], []

    for epoch in range(1, n_epochs + 1):
        if epoch % 20 == 0:
            torch.save(model.state_dict(), f'Resnet38_{epoch=}.pt')
            torch.save(optimizer.state_dict(), f'opt_Resnet38_{epoch=}.pt')
        train_loss, train_acc = train_epoch(model, optimizer, train_loader, loss_func, scheduler)
        val_loss, val_acc = test(model, val_loader, loss_func)
        if scheduler is not None:
            scheduler.step(val_acc)
        
        train_loss_log.append(train_loss)
        train_acc_log.append(train_acc)
        
        val_loss_log.append(val_loss)
        val_acc_log.append(val_acc)
        
#         lasr_lr = scheduler.get_last_lr()[0]

        metrics = {
            "train loss": train_loss, 
            "val loss": val_loss,
            "train acc": train_acc, 
            "val acc": val_acc,
#             "lasr lr": lasr_lr
            
        }

        wandb.log(metrics)
        
        clear_output()
        print(f"Epoch {epoch}")
        print(f" train loss: {train_loss}, train acc: {train_acc}")
        print(f" val loss: {val_loss}, val acc: {val_acc}")
#         if scheduler is not None:
#             print(f" last used lr: {lasr_lr}\n")

    return train_loss_log, train_acc_log, val_loss_log, val_acc_log


## Напишем функцию, которая будет сохранять предсказыванные лейблы:

In [139]:
#!g1.1

def get_test_labels(model):
    model.eval()
    test_labels = []
    
    test_transformer = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5696, 0.5450, 0.4936], [0.2430, 0.2375, 0.2555]),
    ])

    for filename in os.listdir('/home/jupyter/mnt/datasets/bhw1/test/test'):
        image = Image.open(f'/home/jupyter/mnt/datasets/bhw1/test/test/{filename}').convert('RGB')
        image = test_transformer(image).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            logits = model(image)

        test_labels.append({'Id': filename, 'Label': logits.argmax(dim=1).item()})
        
    with open('labels_test.csv', 'w', newline='') as output_file:
        dict_writer = csv.DictWriter(output_file, ('Id', 'Label'))
        dict_writer.writeheader()
        dict_writer.writerows(test_labels)

## Возьмем непредобученный `ResNet50` из `torchvision.models`

In [49]:
#!g1.1

config = {
    "dataset": "bhw1-dataset",
    "machine": "g2.mig",
    "arch": 'ResNet50',
    "weight_decay": 1e-6,
    "optim": "SGD",
}

wandb.init(
    project="image-classification",
    config=config
)

In [50]:
#!g1.1

EPOCHS = 200

model = models.resnet50(num_classes=200, weights=None).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9, weight_decay=1e-6)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS,
)

tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(model, optimizer, EPOCHS, train_loader, val_loader, scheduler, nn.CrossEntropyLoss())

Epoch 200
 train loss: 0.012789208430622239, train acc: 0.996425
 val loss: 2.817658856487274, val acc: 0.615
 last used lr: 0.0



In [51]:
#!g1.1
get_test_labels(model)
torch.save(model.state_dict(), 'Resnet50-part1.pt')
torch.save(optimizer.state_dict(), 'opt-Resnet50-part1.pt')

## Обучаем

In [92]:
#!g1.1

transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.5, 1.0)),
    
    transforms.RandomPerspective(distortion_scale=0.1, p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    
    transforms.RandomApply([
        transforms.ColorJitter(brightness=0.01),
        transforms.ColorJitter(contrast=0.01),
        transforms.ColorJitter(saturation=0.01),
    ], p=0.1),
    
    transforms.RandomApply([
        transforms.GaussianBlur(kernel_size=5),
    ], p=0.5),
    
    transforms.RandomAutocontrast(),
    transforms.RandomEqualize(),
    
    transforms.ToTensor(),
    transforms.Normalize([0.5696, 0.5450, 0.4936], [0.2430, 0.2375, 0.2555]),
    
    transforms.RandomErasing(),
])

train_loader, val_loader = get_loaders(root='/home/jupyter/mnt/datasets/bhw1',  batch_size=32, train_transform=transform, test_size=0.2)

In [93]:
#!g1.1

config = {
    "dataset": "bhw1-dataset",
    "machine": "g2.mig",
    "arch": 'resnet50',
    "weight_decay": 3 * 1e-5,
    "optim": "SGD",
}

wandb.init(
    project="image-classification",
    config=config
)

In [94]:
#!g1.1

EPOCHS = 100

model = models.resnet50(num_classes=200, weights=None).to(DEVICE)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=3 * 1e-5)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
    optimizer,
    T_max=EPOCHS,
)

tr_loss_log, tr_acc_log, val_loss_log, val_acc_log = train(model, optimizer, EPOCHS, train_loader, val_loader, scheduler, nn.CrossEntropyLoss())

Epoch 100
 train loss: 0.35560950281918047, train acc: 0.9056
 val loss: 1.6498496806144713, val acc: 0.6624
 last used lr: 0.0



In [95]:
#!g1.1

get_test_labels(model)
torch.save(model.state_dict(), 'Resnet50-part2.pt')
torch.save(optimizer.state_dict(), 'opt-Resnet50-part2.pt')